<a href="https://colab.research.google.com/github/dksifoua/Question-Answering/blob/master/2%20-%20BiDAF%2C%20Bi-Directional%20Attention%20Flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Nov  2 00:15:26 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Load dependencies

In [2]:
!pip install tqdm --upgrade >> /dev/null 2>&1
!pip install spacy --upgrade >> /dev/null 2>&1
!python -m spacy download en >> /dev/null 2>&1

In [3]:
import re
import json
import time
import tqdm
import spacy
import string
import itertools
import collections
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.core.display import display, HTML

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_sequence, pad_packed_sequence
from torch.utils.data import Dataset, DataLoader

In [4]:
SEED = 546
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {DEVICE}')

Device: cuda


## Prepare data

***Download data***

In [5]:
!rm -rf ./data
!mkdir ./data

!wget --no-check-certificate \
    https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json \
    -O ./data/train-v1.1.json

!wget --no-check-certificate \
    https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json \
    -O ./data/dev-v1.1.json

--2020-11-02 00:15:51--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.110.153, 185.199.111.153, 185.199.108.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30288272 (29M) [application/json]
Saving to: ‘./data/train-v1.1.json’

./data/train-v1.1.j 100%[===================>]  28.88M  60.8MB/s    in 0.5s    

2020-11-02 00:15:53 (60.8 MB/s) - ‘./data/train-v1.1.json’ saved [30288272/30288272]

--2020-11-02 00:15:53--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.110.153, 185.199.111.153, 185.199.108.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4854279 (4.6M) [application/json]
Saving to: ‘./data/dev-v1.1.json’


***Load JSON data***

In [6]:
def load(path):
    with open(path, mode='r', encoding='utf-8') as file:
        return json.load(file)['data']
    raise FileNotFoundError

In [7]:
train_raw_data = load('./data/train-v1.1.json')
valid_raw_data = load('./data/dev-v1.1.json')
print(f'Length of raw train data: {len(train_raw_data):,}')
print(f'Length of raw valid data: {len(valid_raw_data):,}')

Length of raw train data: 442
Length of raw valid data: 48


***Parse JSON data***

In [8]:
def parse(data, nlp=spacy.load('en')):
    qas = []
    for paragraphs in tqdm.tqdm(data):
        for para in paragraphs['paragraphs']:
            context = nlp(para['context'], disable=['parser'])
            for qa in para['qas']:
                id = qa['id']
                question = nlp(qa['question'], disable=['parser', 'tagger', 'ner'])
                for ans in qa['answers']:
                    qas.append({
                        'id': id,
                        'context': context,
                        'question': question,
                        'answer': nlp(ans['text'], disable=['parser', 'tagger', 'ner']),
                        'answer_start': ans['answer_start'],
                    })
    return qas

In [9]:
train_qas = parse(train_raw_data)
valid_qas = parse(valid_raw_data)
print()
print(f'Length of train qa pairs: {len(train_qas):,}')
print(f'Length of valid qa pairs: {len(valid_qas):,}')
print('==================== Example ====================')
print('Id:', train_qas[0]['id'])
print('Context:', train_qas[0]['context'])
print('Question:', train_qas[0]['question'])
print('Answer starts at:', train_qas[0]['answer_start'])
print('Answer:', train_qas[0]['answer'])

100%|██████████| 48/48 [00:34<00:00,  1.40it/s]


Length of train qa pairs: 87,599
Length of valid qa pairs: 34,726
==================== Example ====================
Id: 5733be284776f41900661182
Context: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
Question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Answer starts at: 515
Answer: Saint Bernadette Soubirous


In [10]:
def test_answer_start(qas):
    """Test answer_start are correct in train set"""
    for qa in tqdm.tqdm(qas):
        answer = qa['answer'].text
        context = qa['context'].text
        answer_start = qa['answer_start']
        assert answer == context[answer_start:answer_start + len(answer)]

In [11]:
test_answer_start(train_qas)
test_answer_start(valid_qas)

100%|██████████| 34726/34726 [00:02<00:00, 11754.15it/s]


***Add targets***

In [12]:
def add_targets(qas):
    """Add start and end index token"""
    for qa in qas:
        context = qa['context']
        answer = qa['answer']
        ans_start = qa['answer_start']
        for i in range(len(context)):
            if context[i].idx == ans_start:
                ans = context[i:i + len(answer)]
                qa['target'] = [ans[0].i, ans[-1].i]
                break

In [13]:
%%time
add_targets(train_qas)
add_targets(valid_qas)
print(f'Length of train qa pairs: {len(train_qas):,}')
print(f'Length of valid qa pairs: {len(valid_qas):,}')

Length of train qa pairs: 87,599
Length of valid qa pairs: 34,726
CPU times: user 1.5 s, sys: 22 ms, total: 1.52 s
Wall time: 1.53 s


In [14]:
def filter_qas(qa):
    """Remove bad targets"""
    if 'target' in [*qa.keys()]:
        start, end = qa['target']
        return qa['context'][start:end + 1].text == qa['answer'].text
    return False

In [15]:
%%time
train_qas = [*filter(filter_qas, train_qas)]
valid_qas = [*filter(filter_qas, valid_qas)]
print(f'Length of train qa pairs after filtering out bad qa pairs: {len(train_qas):,}')
print(f'Length of valid qa pairs after filtering out bad qa pairs: {len(valid_qas):,}')

Length of train qa pairs after filtering out bad qa pairs: 86,597
Length of valid qa pairs after filtering out bad qa pairs: 34,295
CPU times: user 1.16 s, sys: 2.01 ms, total: 1.16 s
Wall time: 1.16 s


In [16]:
def test_targets(qas):
    for qa in qas:
        if 'target' in [*qa.keys()]:
            start, end = qa['target']
            assert qa['context'][start:end + 1].text == qa['answer'].text

In [17]:
%%time
test_targets(train_qas)
test_targets(valid_qas)

CPU times: user 1.09 s, sys: 957 µs, total: 1.09 s
Wall time: 1.1 s


***Build vocabularies***

In [18]:
class Vocab:

    def __init__(self, pad_token, unk_token):
        self.pad_token = pad_token
        self.unk_token = unk_token
        self.vocab = None
        self.word2count = None
        self.word2index = None
        self.index2word = None
    
    def build(self, data, min_freq):
        """
        :param List[Union[spacy.tokens.doc.Doc, str]] data
        :param int min_freq
        """
        words = [self.pad_token, self.unk_token]
        type_0 = type(data[0])
        if type_0 == spacy.tokens.doc.Doc:
            for item in data: # context and question
                words += [word.text.lower() for word in item]
        elif type_0 == str: # id
            words += data
        self.word2count = collections.Counter(words)
        self.vocab = sorted(filter(
            lambda word: self.word2count[word] >= min_freq or word == self.pad_token or word == self.unk_token, self.word2count
        ))
        self.word2index = {word: index for index, word in enumerate(self.vocab)}
        self.index2word = {index: word for index, word in enumerate(self.vocab)}
    
    def __len__(self):
        return len(self.vocab)
    
    def stoi(self, word):
        return self.word2index.get(str(word), self.word2index[self.unk_token])

    def itos(self, index):
        return self.index2word[index]

In [19]:
class CharVocab:

    def __init__(self, pad_token, unk_token):
        self.pad_token = pad_token
        self.unk_token = unk_token
        self.vocab = None
        self.char2count = None
        self.char2index = None
        self.index2char = None
    
    def build(self, data, min_freq):
        """
        :param List[Union[spacy.tokens.doc.Doc, str, Tuple]] data
        :param int min_freq
        """
        chars = [self.pad_token, self.unk_token]
        type_0 = type(data[0])
        if type_0 == spacy.tokens.doc.Doc:
            for item in data: # context and question
                for word in item:
                    chars += [*word.text.lower().strip()]
        else:
            raise Exception
        self.char2count = collections.Counter(chars)
        self.vocab = sorted(filter(
            lambda char: self.char2count[char] >= min_freq or char == self.pad_token or char == self.unk_token, self.char2count
        ))
        self.char2index = {char: index for index, char in enumerate(self.vocab)}
        self.index2char = {index: char for index, char in enumerate(self.vocab)}
    
    def __len__(self):
        return len(self.vocab)
    
    def stoi(self, char):
        return self.char2index.get(str(char), self.char2index[self.unk_token])

    def itos(self, index):
        return self.index2char[index]

In [20]:
%%time
PAD_TOKEN = '<pad>'
UNK_TOKEN = '<unk>'

ID = Vocab(pad_token=PAD_TOKEN, unk_token=UNK_TOKEN)
TEXT = Vocab(pad_token=PAD_TOKEN, unk_token=UNK_TOKEN)
CHAR = CharVocab(pad_token=PAD_TOKEN, unk_token=UNK_TOKEN)

ids = [*map(lambda qa: qa['id'], train_qas)] + [*map(lambda qa: qa['id'], valid_qas)]
contexts, questions = zip(*map(lambda qa: (qa['context'], qa['question']), train_qas))

ID.build(data=[*set(ids)], min_freq=0)
TEXT.build(data=[*set(contexts)] + [*set(questions)], min_freq=5)
CHAR.build(data=[*set(contexts)] + [*set(questions)], min_freq=0)

print(f'Length of ID vocabulary: {len(ID):,}')
print(f'Length of TEXT vocabulary: {len(TEXT):,}')
print(f'Length of CHAR vocabulary: {len(CHAR):,}')

Length of ID vocabulary: 97,108
Length of TEXT vocabulary: 26,885
Length of CHAR vocabulary: 1,261
CPU times: user 9.44 s, sys: 271 ms, total: 9.71 s
Wall time: 9.71 s


***Build datasets***

In [21]:
class SQuADV1Dataset(Dataset):

    def __init__(self, data, id_vocab, text_vocab, char_vocab):
        self.data = data
        self.id_vocab = id_vocab
        self.text_vocab = text_vocab
        self.char_vocab = char_vocab
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        
        id = torch.LongTensor([self.id_vocab.stoi(item['id'])])

        context = item['context']
        ctx = torch.LongTensor([*map(lambda word: self.text_vocab.stoi(word.text.lower()), context)])
        char_ctx = [*map(lambda word: [*map(self.char_vocab.stoi, [*word.text.lower().strip()])], context)]
        char_len_ctx = torch.LongTensor([*map(len, char_ctx)])
        char_ctx = torch.LongTensor([*itertools.chain.from_iterable(char_ctx)])
        
        question = item['question']
        qst = torch.LongTensor([*map(lambda word: self.text_vocab.stoi(word.text.lower()), question)])
        char_qst = [*map(lambda word: [*map(self.char_vocab.stoi, [*word.text.lower().strip()])], question)]
        char_len_qst = torch.LongTensor([*map(len, char_qst)])
        char_qst = torch.LongTensor([*itertools.chain.from_iterable(char_qst)])
        
        trg = torch.LongTensor(item['target'])
        
        return id, ctx, char_ctx, char_len_ctx, qst, char_qst, char_len_qst, trg

In [22]:
train_dataset = SQuADV1Dataset(data=train_qas, id_vocab=ID, text_vocab=TEXT, char_vocab=CHAR)
valid_dataset = SQuADV1Dataset(data=valid_qas, id_vocab=ID, text_vocab=TEXT, char_vocab=CHAR)

id, ctx, char_ctx, char_len_ctx, qst, char_qst, char_len_qst, trg = train_dataset[0]
print(f'id shape: {id.shape}')
print(f'ctx shape: {ctx.shape}')
print(f'char_ctx shape: {char_ctx.shape}')
print(f'char_len_ctx shape: {char_len_ctx.shape}')
print(f'qst shape: {qst.shape}')
print(f'char_qst shape: {char_qst.shape}')
print(f'char_len_qst shape: {char_len_qst.shape}')
print(f'trg shape: {trg.shape}')

id shape: torch.Size([1])
ctx shape: torch.Size([142])
char_ctx shape: torch.Size([572])
char_len_ctx shape: torch.Size([142])
qst shape: torch.Size([14])
char_qst shape: torch.Size([59])
char_len_qst shape: torch.Size([14])
trg shape: torch.Size([2])


***Build data loaders***

In [23]:
class DotDict(dict):
    __getattr__ = dict.get

In [24]:
def pad_char_sequence(batch_padded_seq, batch_char_seq, batch_char_len, pad_index):
    """
    :param Tensor[batch_size, seq_len] batch_padded_seq
    :param ListTensor[batch_size, seq_len * n_chars] batch_char_seq
    :param ListTensor[batch_size, seq_len] batch_char_len
    """
    max_char_len = 0
    for char_len in batch_char_len:
        m = char_len.max().item()
        if m > max_char_len:
            max_char_len = m
    batch_padded_char = torch.ones((batch_padded_seq.size(0), batch_padded_seq.size(1), max_char_len)) * pad_index
    for i in range(batch_padded_seq.size(0)):
        seq = batch_padded_seq[i] # [seq_len]
        char_seq = batch_char_seq[i] # [seq_len * n_chars]
        char_len = batch_char_len[i] # [seq_len]
        j = 0
        for k, length in enumerate(char_len):
            batch_padded_char[i, k, j:length] = char_seq[j:length]
            j += length
    return batch_padded_char

In [25]:
def add_padding(batch, text_vocab=TEXT, char_vocab=CHAR, include_lengths=True, device=DEVICE):
    """Pad batch of sequence with different lengths"""
    batch_id, batch_ctx, batch_char_ctx, batch_char_len_ctx, batch_qst, batch_char_qst, batch_char_len_qst, batch_trg = zip(*batch)
    if include_lengths:
        len_ctx = torch.LongTensor([ctx.size(0) for ctx in batch_ctx]).to(device)
        len_qst = torch.LongTensor([qst.size(0) for qst in batch_qst]).to(device)
    batch_padded_id = pad_sequence(batch_id, batch_first=True).to(device)
    batch_padded_ctx = pad_sequence(batch_ctx, batch_first=True, padding_value=text_vocab.stoi(text_vocab.pad_token)).to(device)
    batch_padded_qst = pad_sequence(batch_qst, batch_first=True, padding_value=text_vocab.stoi(text_vocab.pad_token)).to(device)
    batch_padded_trg = pad_sequence(batch_trg, batch_first=True).to(device)

    pad_index = char_vocab.stoi(text_vocab.pad_token)
    batch_padded_char_ctx = pad_char_sequence(batch_padded_seq=batch_padded_ctx,
                                              batch_char_seq=batch_char_ctx,
                                              batch_char_len=batch_char_len_ctx,
                                              pad_index=pad_index)
    batch_padded_char_qst = pad_char_sequence(batch_padded_seq=batch_padded_qst,
                                              batch_char_seq=batch_char_qst,
                                              batch_char_len=batch_char_len_qst,
                                              pad_index=pad_index)
    return DotDict({
        'id': batch_padded_id,
        'ctx': (batch_padded_ctx, len_ctx) if include_lengths else batch_padded_ctx,
        'char_ctx': batch_padded_char_ctx,
        'qst': (batch_padded_qst, len_qst) if include_lengths else batch_padded_qst,
        'char_qst': batch_padded_char_qst,
        'trg': batch_padded_trg
    })

In [26]:
BATCH_SIZE = 32

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=add_padding)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, collate_fn=add_padding)

for batch in train_dataloader:
    print('batch.id shape:', batch.id.shape)
    print('batch.ctx shape:', batch.ctx[0].shape, batch.ctx[1].shape)
    print('batch.char_ctx shape:', batch.char_ctx.shape)
    print('batch.qst shape:', batch.qst[0].shape, batch.qst[1].shape)
    print('batch.char_qst shape:', batch.char_qst.shape)
    print('batch.trg shape:', batch.trg.shape)
    break

batch.id shape: torch.Size([32, 1])
batch.ctx shape: torch.Size([32, 253]) torch.Size([32])
batch.char_ctx shape: torch.Size([32, 253, 15])
batch.qst shape: torch.Size([32, 19]) torch.Size([32])
batch.char_qst shape: torch.Size([32, 19, 14])
batch.trg shape: torch.Size([32, 2])


***TODO: Download pretrained GloVe embedding***

## Modeling

***Character Embedding Layer***

In [28]:
class CharacterEmbeddingLayer(nn.Module):

    def __init__(self, char_vocab_size, char_embedding_size, token_embedding_size, kernel_size, pad_index):
        super(CharacterEmbeddingLayer, self).__init__()
        self.char_vocab_size = char_vocab_size
        self.char_embedding_size = char_embedding_size
        self.token_embedding_size = token_embedding_size
        self.kernel_size = kernel_size
        self.pad_index = pad_index
        self.embedding = nn.Embedding(vocab_size, char_embedding_size, padding_idx=pad_index)
        self.cond2d = nn.Conv2d(1, token_embedding_size, kernel_size=(char_embedding_size, kernel_size))
    
    def forward(self, char_sequences):
        """
        :param LongTensor[batch_size, seq_len, char_len]
        :return FloatTensor[batch_size, seq_len, token_embedding_size]
        """
        batch_size, seq_len, char_len = char_sequences.size()
        embedded = self.embedding(char_sequences) # [batch_size, seq_len, char_len, char_embedding_size]
        embedded = embedded.transpose(-1, -2) # [batch_size, seq_len, char_embedding_size, char_len]
        embedded = embedded.view(batch_size * seq_len, self.char_embedding_size, char_len) # [batch_size * seq_len, char_embedding_size, char_len]
        embedded = embedded.unsqueeze(1) # [batch_size * seq_len, 1, char_embedding_size, char_len]
        conved = self.cond2d(embedded) # [batch_size * seq_len, token_embedding_size, 1, char_len - kernel_size + 1]
        # May be apply ReLU non-linearity???
        conved = conved.squeeze(2) # [batch_size * seq_len, token_embedding_size, char_len - kernel_size + 1]
        out = F.max_pool2d(conved, kernel_size=conved.size(-1)) # [batch_size * seq_len, token_embedding_size, 1]
        out = out.squeeze(-1) # [batch_size * seq_len, token_embedding_size]
        out = out.view(batch_size, seq_len, self.token_embedding_size) # [batch_size, seq_len, token_embedding_size]
        # May be apply bias + tanh non linearity???
        return out

***Highway Network Layer***

[Highway Networks paper](https://arxiv.org/pdf/1505.00387.pdf)

In [31]:
class HighwayNetworkLayer(nn.Module):

    def __init__(self, hidden_size, n_layers):
        super(HighwayNetworkLayer, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.fc_flow = nn.ModuleList([nn.Linear(hidden_size, hidden_size) for _ in range(n_layers)])
        self.fc_gate = nn.ModuleList([nn.Linear(hidden_size, hidden_size) for _ in range(n_layers)])
    
    def forward(self, x):
        """
        :param FloatTensor[batch_size, seq_len, hidden_size] x
        :return FloatTensor[batch_size, seq_len, hidden_size]
        """
        for i in range(self.n_layers):
            flow = F.relu(self.fc_flow[i](x))
            gate = F.sigmoid(self.fc_gate[i](x))
            x = gate * flow + (1 - gate) * x
        return x

***Contextual Embedding Layer***

In [32]:
class ContextualEmbeddingLayer(nn.Module):

    def __init__(self, hidden_size, n_layers):
        super(ContextualEmbeddingLayer, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.high_network_layer = HighwayNetworkLayer(hidden_size=hidden_size, n_layers=n_layers)
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True, bidirectional=True)

    def forward(self, inputs, seq_len):
        """
        :param FloatTensor[batch_size, seq_len, hidden_size] inputs
        :param FloatTensor[batch_size, seq_len] seq_len
        :return FloatTensor[batch_size, seq_len, hidden_size * 2]
        """
        highway = self.high_network_layer(inputs) # [batch_size, seq_len, hidden_size]
        packed = pack_padded_sequence(highway, seq_len, batch_first=True, enforce_sorted=False)
        out_packed, _ = self.lstm(packed)
        out_padded, _ = pad_packed_sequence(out_packed, batch_first=True) # [batch_size, seq_len, hidden_size * 2]
        return out_padded

***Attention Flow Layer***